In [1]:
import sys
print(sys.path)
print(sys.executable)
print(sys.version)
print(sys.version_info)

# import os
# memory = '3g'
# pyspark_submit_args = ' --driver-memory ' + memory + " --executor-memory " + "3g" +  ' pyspark-shell' 
# os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

['/opt/spark-data/CW', '/opt/bitnami/spark/python/lib/py4j-0.10.9.3-src.zip', '/opt/bitnami/spark/python', '/opt/bitnami/spark/python/build', '/opt/bitnami/python/lib/python38.zip', '/opt/bitnami/python/lib/python3.8', '/opt/bitnami/python/lib/python3.8/lib-dynload', '', '/opt/bitnami/python/lib/python3.8/site-packages', '/opt/bitnami/python/lib/python3.8/site-packages/pip-21.3.1-py3.8.egg']
/opt/bitnami/python/bin/python3
3.8.13 (default, Mar 25 2022, 21:22:37) 
[GCC 8.3.0]
sys.version_info(major=3, minor=8, micro=13, releaselevel='final', serial=0)


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,StringType
from pyspark.pandas.config import set_option, reset_option

conf =SparkConf().setMaster("local[*]").setAppName("CW") 
SparkContext.setSystemProperty('spark.executor.memory', '1g')
SparkContext.setSystemProperty('spark.driver.memory', '1g')

sc = SparkContext(conf = conf)
spark = SparkSession(sc)
sc._conf.getAll()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 13:39:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/30 13:39:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


[('spark.driver.host', 'master'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '1g'),
 ('spark.app.id', 'local-1651325997673'),
 ('spark.executor.memory', '1g'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'CW'),
 ('spark.app.startTime', '1651325996759'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '32991'),
 ('spark.ui.showConsoleProgress', 'true')]

In [3]:
path = "./Postures.csv"
df = spark.read.option("header",True).option("nullValue","?").csv(path)
# df.show()
# df.na.fill(0).show()

In [4]:
df.printSchema()
# df.dtypes
# the filling type must be consistent with the original type. Thus, filling string here.
df=df.na.fill('0')

root
 |-- Class: string (nullable = true)
 |-- User: string (nullable = true)
 |-- X0: string (nullable = true)
 |-- Y0: string (nullable = true)
 |-- Z0: string (nullable = true)
 |-- X1: string (nullable = true)
 |-- Y1: string (nullable = true)
 |-- Z1: string (nullable = true)
 |-- X2: string (nullable = true)
 |-- Y2: string (nullable = true)
 |-- Z2: string (nullable = true)
 |-- X3: string (nullable = true)
 |-- Y3: string (nullable = true)
 |-- Z3: string (nullable = true)
 |-- X4: string (nullable = true)
 |-- Y4: string (nullable = true)
 |-- Z4: string (nullable = true)
 |-- X5: string (nullable = true)
 |-- Y5: string (nullable = true)
 |-- Z5: string (nullable = true)
 |-- X6: string (nullable = true)
 |-- Y6: string (nullable = true)
 |-- Z6: string (nullable = true)
 |-- X7: string (nullable = true)
 |-- Y7: string (nullable = true)
 |-- Z7: string (nullable = true)
 |-- X8: string (nullable = true)
 |-- Y8: string (nullable = true)
 |-- Z8: string (nullable = true)
 |--

In [5]:
df = df.where(df['Class']!=0)

In [7]:
df = df.select('Class','X0','Y0','Z0','X1','Y1','Z1','X2','Y2','Z2','X3','Y3','Z3','X4','Y4','Z4')
df.printSchema()

root
 |-- Class: string (nullable = false)
 |-- X0: string (nullable = false)
 |-- Y0: string (nullable = false)
 |-- Z0: string (nullable = false)
 |-- X1: string (nullable = false)
 |-- Y1: string (nullable = false)
 |-- Z1: string (nullable = false)
 |-- X2: string (nullable = false)
 |-- Y2: string (nullable = false)
 |-- Z2: string (nullable = false)
 |-- X3: string (nullable = false)
 |-- Y3: string (nullable = false)
 |-- Z3: string (nullable = false)
 |-- X4: string (nullable = false)
 |-- Y4: string (nullable = false)
 |-- Z4: string (nullable = false)



## Convert String to Float64

In [8]:
df2 = df.toPandas()
df2 = df2.astype('float')
df2.dtypes

Class    float64
X0       float64
Y0       float64
Z0       float64
X1       float64
Y1       float64
Z1       float64
X2       float64
Y2       float64
Z2       float64
X3       float64
Y3       float64
Z3       float64
X4       float64
Y4       float64
Z4       float64
dtype: object

## Mean Filling of Missing Value

In [9]:
meanList = []
for i in range(1,6):
    meanList.append(df2[(df2["Class"] == i) & (df2["X3"] != 0)]["X3"].mean())
def fill_val1(col):
    Class = col[0]
    axis = col[1]
    if (axis == 0):
        
        if Class == 1:
            return meanList[0]
        
        elif Class == 2:
            return meanList[1]
        elif Class == 3:
            return meanList[2]
        elif Class == 4:
            return meanList[3]
        elif Class == 5:
            return meanList[4]
    else:
        return axis

In [10]:
meanList2 = []
for i in range(1,6):
    meanList2.append(df2[(df2["Class"] == i) & (df2["Y3"] != 0)]["Y3"].mean())
def fill_val2(col):
    Class = col[0]
    axis = col[1]
    if (axis == 0):
        
        if Class == 1:
            return meanList2[0]
        
        elif Class == 2:
            return meanList2[1]
        elif Class == 3:
            return meanList2[2]
        elif Class == 4:
            return meanList2[3]
        elif Class == 5:
            return meanList2[4]
    else:
        return axis

In [11]:
meanList3 = []
for i in range(1,6):
    meanList3.append(df2[(df2["Class"] == i) & (df2["Z3"] != 0)]["Z3"].mean())
def fill_val3(col):
    Class = col[0]
    axis = col[1]
    if (axis == 0):

        if Class == 1:
            return meanList3[0]
        
        elif Class == 2:
            return meanList3[1]
        elif Class == 3:
            return meanList3[2]
        elif Class == 4:
            return meanList3[3]
        elif Class == 5:
            return meanList3[4]
    else:
        return axis

In [12]:
def getmeanList(colName):
    meanList = []
    for i in range(1,6):
        meanList.append(df2[(df2["Class"] == i) & (df2[colName] != 0)][colName].mean())
    return meanList
def fill_val(meanList,col):
    Class = col[0]
    axis = col[1]
    if (axis == 0):
        if Class == 1:
            return meanList[0]

        elif Class == 2:
            return meanList[1]
        elif Class == 3:
            return meanList[2]
        elif Class == 4:
            return meanList[3]
        elif Class == 5:
            return meanList[4]
    else:
        return axis

In [13]:
import functools

df2['X3'] = df2[['Class', "X3"]].apply(fill_val1, axis = 1)
df2['Y3'] = df2[['Class', "Y3"]].apply(fill_val2, axis = 1)
df2['Z3'] = df2[['Class', "Z3"]].apply(fill_val3, axis = 1)

meanfill=getmeanList('X4')
func=functools.partial(fill_val,meanfill)
df2['X4'] = df2[['Class', "X4"]].apply(func, axis = 1)

meanfill=getmeanList('Y4')
func=functools.partial(fill_val,meanfill)
df2['Y4'] = df2[['Class', "Y4"]].apply(func, axis = 1)

meanfill=getmeanList('Z4')
func=functools.partial(fill_val,meanfill)
df2['Z4'] = df2[['Class', "Z4"]].apply(func, axis = 1)

In [14]:
df=spark.createDataFrame(df2) 
df.printSchema()

root
 |-- Class: double (nullable = true)
 |-- X0: double (nullable = true)
 |-- Y0: double (nullable = true)
 |-- Z0: double (nullable = true)
 |-- X1: double (nullable = true)
 |-- Y1: double (nullable = true)
 |-- Z1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- Y2: double (nullable = true)
 |-- Z2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- Y3: double (nullable = true)
 |-- Z3: double (nullable = true)
 |-- X4: double (nullable = true)
 |-- Y4: double (nullable = true)
 |-- Z4: double (nullable = true)



## Save to csv

In [15]:
# df.write.csv("/opt/spark-data/CW/new_feature2.csv")

df.toPandas().to_csv('new_feature2.csv',index=False)
df = df.select('Class','X0','Y0','Z0','X1','Y1','Z1','X2','Y2','Z2','X3','Y3','Z3')
df.toPandas().to_csv('new_feature.csv',index=False)

# df.summary().show()

In [16]:
# to speed up, uncomment this and run directly
# path = "./new_feature.csv"
# df = spark.read.option("header",True).csv(path)
# df = df.to_pandas_on_spark()
# df = df.astype('float')
# print(df.dtypes)
# df = df.to_spark()

In [18]:
import numpy as np
from numpy import allclose
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors
from  pyspark.ml.classification import RandomForestClassifier

assembler = VectorAssembler(inputCols=df.drop('Class').columns,outputCol="features")
df=assembler.transform(df)
df.select('features').head(1)

22/04/30 13:42:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(features=DenseVector([54.2639, 71.4668, -64.8077, 76.8956, 42.4625, -72.7805, 36.6212, 81.6806, -52.9193, 85.2323, 67.7492, -73.6841, 59.1886, 10.6789, -71.2978]))]

In [18]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train, test = df.randomSplit([0.7, 0.3], seed=12345)

# X_train = train.drop('ClassIndex')
# y_train = train.select('ClassIndex')
# X_test = test.drop('ClassIndex')
# y_test = test.select('ClassIndex')
# print("The number of rows in training set is: ",X_train.count(),". The number of columns in training set is ",len(X_train.columns))
# print("The number of rows in testing is: ",X_test.count(),". The number of columns in testing set is ",len(X_test.columns))

# from sklearn.ensemble import RandomForestClassifier
# rtree = RandomForestClassifier(100)
# rtree.fit(X_train, y_train)
# prediction = rtree.predict(X_test)
rf = RandomForestClassifier(numTrees=100,maxDepth=10,maxBins=32,labelCol = 'ClassIndex',featuresCol = 'features') # maxDepth=2, labelCol="indexed", seed=42, leafCol="leafId"
rf_model = rf.fit(train)
rf_predictions = rf_model.transform(test)

22/04/29 15:06:50 WARN TaskSetManager: Stage 6 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 15:06:50 WARN TaskSetManager: Stage 7 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 15:06:51 WARN TaskSetManager: Stage 9 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 15:06:52 WARN TaskSetManager: Stage 11 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 15:06:53 WARN TaskSetManager: Stage 13 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 15:06:53 WARN TaskSetManager: Stage 15 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 15:06:54 WARN TaskSetManager: Stage 17 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.
22/04/29 

In [19]:
# from sklearn.metrics import classification_report,confusion_matrix
# print(classification_report(y_test,prediction))

from pyspark.ml import Pipeline

# Binary Classfication
# from pyspark.ml.evaluation import BinaryClassificationEvaluator
# binary_evaluator = BinaryClassificationEvaluator(labelCol = 'ClassIndex')
# print('Random Forest:' , binary_evaluator.evaluate(rf_predictions))

# Multiclass Classification
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'ClassIndex', metricName = 'accuracy')
print(multi_evaluator.getMetricName())
print('Random Forest Accu:', multi_evaluator.evaluate(rf_predictions))

accuracy


22/04/29 15:07:31 WARN DAGScheduler: Broadcasting large task binary with size 12.7 MiB
22/04/29 15:07:31 WARN TaskSetManager: Stage 29 contains a task of very large size (1033 KiB). The maximum recommended task size is 1000 KiB.


Random Forest Accu: 0.9130564050288821


In [19]:
# y_true = predictions_train.select(['ClassIndex']).collect()
# y_pred = predictions_train.select(['prediction']).collect()
# from sklearn.metrics import classification_report, confusion_matrix
# print(classification_report(y_true, y_pred))
# print(confusion_matrix(y_true,y_pred))

from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
preds_and_labels = rf_predictions.select(['prediction','ClassIndex']).withColumn('label', F.col('ClassIndex').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

22/04/02 12:48:35 WARN DAGScheduler: Broadcasting large task binary with size 11.9 MiB
22/04/02 12:48:36 WARN DAGScheduler: Broadcasting large task binary with size 11.9 MiB
/opt/bitnami/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/04/02 12:48:39 WARN DAGScheduler: Broadcasting large task binary with size 11.9 MiB
22/04/02 12:48:39 WARN DAGScheduler: Broadcasting large task binary with size 11.9 MiB


[[4486.   70.   88.   85.  135.]
 [  35. 4822.   18.   13.    8.]
 [ 156.    6. 4311.  252.   55.]
 [ 172.   16.  229. 3968.  161.]
 [ 560.   26.   61.  237. 3574.]]


In [20]:
spark.stop()
sc.stop()